In [ ]:
import os
from google.cloud import bigquery
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Mean Absolute Error(MAE)用
from sklearn.metrics import mean_absolute_error
# Root Mean Squared Error(RMSE)用
from sklearn.metrics import mean_squared_error

# 環境変数に設定
cwd = os.getcwd()
credentials = 'npsdev-manage-project-b86ba8a617d2.json'
key_path = os.path.join(cwd,"credentials",credentials)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_path

In [ ]:
project_id = 'npsdev-manage-project'
dataset_id = 'share_dataset'
table_id = 'junction_hn_nps_file_regist_count_per_minute'

client = bigquery.Client(project=project_id)

In [ ]:
def add_leap_year_day(input_df):

    day_0229 = 59
    dfA = input_df[0:day_0229]
    dfB = pd.Series(['Nan'])
    dfC = input_df[day_0229:]
    #print(dfA)
    #print(dfB)
    #print(dfC)
    df = pd.concat([dfA,dfB,dfC])
    df = df.set_axis([i for i in range(366)], axis="index")
    df = df.drop(df.columns[[2]], axis=1)

    #print(df[50:70])
    df = df.fillna(method='ffill')

    return df

In [ ]:
def query_bigquery_job(dataset_id,table_id,start_date,end_date,year):
    sql = f"""
    SELECT CAST(registration_date AS DATE) day,
    SUM(registration_count)
    FROM {dataset_id}.{table_id}
    WhERE registration_date >= '{start_date}' AND registration_date <= '{end_date}'
    GROUP BY day
    ORDER BY day
    """
    #print(sql)
    df = client.query(sql).to_dataframe()
    df = df.rename(columns={'day': f'day-{year}','f0_': f'count-{year}'})
    return df

In [ ]:
def plt_show_days (start_date,end_date,leap_year,year):

    df = query_bigquery_job(dataset_id,table_id,start_date,end_date,year)

    if not leap_year:
        df = add_leap_year_day(df)
 
    #print(df[50:70])
    df = df.fillna(method='ffill')

    #display(sql)
    plt.ylabel("printVol")
    plt.plot(df[f'day-{year}'], df[f'count-{year}'], label=year)
    plt.legend(loc = 'upper left')
    plt.show()

    return df


In [ ]:
def plot_show_marge(sr_xplot,sr_yplot_1,labdel_1,sr_yplot_2,labdel_2):

    #可視化
    plt.plot(sr_xplot, sr_yplot_1, label=labdel_1, color ="red")
    plt.plot(sr_xplot, sr_yplot_2, label=labdel_2, color ="blue")
    plt.ylabel("daily_demand")
    plt.xlabel("date")
    plt.legend(loc = 'upper right')
    plt.show()

In [ ]:
#移動平均法
def calc_rooling_mean(input_sr,window_size = 3):
    rm_sr = input_sr.rolling(window = window_size).mean()
    rm_sr = rm_sr.fillna(rm_sr.median())
    return rm_sr

In [ ]:
##指数平滑法（EWM：Exponentially weighted method）
def calc_ewm(input_sr,alpha = 0.5):
    ewm_sr = input_sr.ewm(alpha=alpha).mean()
    ewm_sr = ewm_sr.fillna(ewm_sr.median())
    return ewm_sr

日時集計

In [ ]:
#df_2018 = plt_show_days('2018-01-01','2018-12-31',False,"2018")
df_2019 = plt_show_days('2019-01-01','2019-12-31',False,"2019")
df_2020 = plt_show_days('2020-01-01','2020-12-31',True,"2020")
#df_2021 = plt_show_days('2021-01-01','2021-12-31',False,"2021")

df_concat = pd.concat([df_2019,df_2020],axis=1)
#df_concat


In [ ]:
def calc_accuracy(true_sr , pred_sr):

    #二乗平方根誤差（RMSE：Root Mean Square Error）
    rmse = np.sqrt(mean_squared_error(true_sr, pred_sr))
    print('RMSE : {:.3f}'.format(rmse))

    #平均絶対誤差（MAE：Mean Absolute Error）
    mae = mean_absolute_error(true_sr, pred_sr)
    print('MAE : {:.3f}'.format(mae))

    return {
        "RMSE" : rmse,
        "MAE" : mae
    }


In [ ]:
rm_2019 = calc_rooling_mean(df_concat["count-2019"])
ewm_2019 = calc_ewm(df_concat["count-2019"])
calc_accuracy(df_concat["count-2020"],rm_2019)
calc_accuracy(df_concat["count-2020"],ewm_2019)

plot_show_marge(df_concat["day-2020"],df_concat["count-2020"],"2020-true",ewm_2019,"2020-pred")


In [ ]:
#NeuralProphetをインポート
from neuralprophet import NeuralProphet
#データセットの準備
df1 = pd.read_csv("train.csv")
df1 = df1.query('store == "1" & item == "1"')
df1 = df1.drop(["store", "item"], axis=1)
df1 = df1.rename(columns = {"date": "ds", "sales": "y"})
#データセットの分割
train = df1[:-365]#2013-2016年の販売実績
test = df1[-365:]#2017年の販売実績
#機械学習モデルのインストタンス作成
m = NeuralProphet(seasonality_mode='multiplicative')
#validationの設定(3:1)&モデルのトレーニング
metrics = m.fit(train,freq="D")
#作ったモデルで２０１７年のSalesを予測(trainデータの先３６５日分を予測)
future = m.make_future_dataframe(train, periods=365)
forecast = m.predict(future)
#予測結果をtestのデータフレームに追加
test["pred"] = forecast["yhat1"].to_list()
#MAEで精度を確認
from sklearn.metrics import mean_absolute_error
print('MAE(NeuralProphet):')
print(mean_absolute_error(test['y'], test['pred']))#3.9889207134508107
#可視化 
test.plot(title='Forecast evaluation',ylim=[0,50])

月次集計

In [ ]:
sql = f"""
SELECT DATE_TRUNC(CAST(registration_date AS DATE), MONTH) AS month,
    COUNT(registration_date)
FROM {dataset_id}.{table_id}
WhERE registration_date >='2020-01-01' AND registration_date <='2020-12-31'
GROUP BY month
ORDER BY month
"""

df = client.query(sql).to_dataframe()
display(df)
